In [1]:
import os
import sys
import tarfile
import time
import urllib.request

source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'

In [2]:
def reporthook(count, block_size, total_size):
  global start_time
  if count == 0:
    start_time = time.time()
    return
  duration = time.time() - start_time
  progress_size = int(count * block_size)
  speed = progress_size / (1024.**2 * duration)
  percent = count * block_size * 100. / total_size

  sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
   (percent, progress_size / (1024.**2), speed, duration))

  sys.stdout.flush()

if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
  urllib.request.urlretrieve(source, target, reporthook)

100% | 80 MB | 2.40 MB/s | 33 sec elapsed

In [17]:
if not os.path.isdir('aclimdb'):
  with tarfile.open(target, 'r:gz') as tar:
    tar.extractall()

In [4]:
!pip install pyprind

In [18]:
import pyprind
import pandas as pd
import os

In [19]:
basepath ='aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)

In [68]:
data = []
for s in ('test', 'train'):
  for l in ('pos', 'neg'):
    path = os.path.join(basepath, s, l)
    for file in sorted(os.listdir(path)):
      with open(os.path.join(path, file),
                'r', encoding='utf-8') as infile:
                  txt = infile.read()
    data.append({'review': txt, 'sentiment' : labels[l]})
    pbar.update()

df = pd.DataFrame(data)

In [69]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [70]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,Working-class romantic drama from director Mar...,1
1,"This is one of the dumbest films, I've ever se...",0
2,David Bryce's comments nearby are exceptionall...,0


In [71]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two'])

bag = count.fit_transform(docs)

print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [72]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [73]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


In [74]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

tf-idf of term "is" = 3.00


In [75]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf

array([3.38629436, 3.        , 3.38629436, 1.28768207, 1.28768207,
       1.28768207, 2.        , 1.69314718, 1.28768207])

In [76]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0.50238645, 0.44507629, 0.50238645, 0.19103892, 0.19103892,
       0.19103892, 0.29671753, 0.25119322, 0.19103892])

In [77]:
df.loc[0, 'review'][-50:]

import re
def preprocessor(text):
  text = re.sub('<[^>]*>','', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                         text)

  text = (re.sub('[\W]+', ' ', text.lower()) +
          ' '.join(emoticons).replace('-',''))

  return text

preprocessor(df.loc[0, 'review'][-50:])

' really be satisfying any other way from '

In [78]:
df['review'] = df['review'].apply(preprocessor)

In [79]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
  return text.split()

def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

In [80]:
tokenizer('runners like running and thus they run')

tokenizer_porter('runners like running and thus they run')

import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

In [105]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [106]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (4,), y_train shape: (4,)
X_test shape: (0,), y_test shape: (0,)


In [94]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

In [95]:
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
                {'vect__ngram_range': [(1, 1)],
                 'vect__stop_words': [stop, None],
                 'vect__tokenizer': [tokenizer, tokenizer_porter],
                 'vect__use_idf':[False],
                 'vect__norm':[None],
                 'clf__penalty': ['l1', 'l2'],
                 'clf__C': [1.0, 10.0, 100.0]},
              ]

In [96]:
lr_tfidf = Pipeline([('vect', tfidf),
 ('clf', LogisticRegression(random_state=1, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=2,n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)
print('CV 정확도: %.3f' % gs_lr_tfidf.best_score_)

clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

최적의 매개변수 조합: {'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few',

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: Found array with 0 sample(s) (shape=(0, 413)) while a minimum of 1 is required by TfidfTransformer.

In [88]:
# TF-IDF 벡터라이저와 로지스틱 회귀를 사용한 파이프라인
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
                {'vect__ngram_range': [(1, 1)],
                 'vect__stop_words': [stop, None],
                 'vect__tokenizer': [tokenizer, tokenizer_porter],
                 'vect__use_idf':[False],
                 'vect__norm':[None],
                 'clf__penalty': ['l1', 'l2'],
                 'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=1, solver='liblinear'))])

# 교차 검증 및 그리드 서치
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=2, n_jobs=-1)

# 모델 훈련
gs_lr_tfidf.fit(X_train, y_train)

# 최적의 매개변수 및 정확도 출력
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)
print('CV 정확도: %.3f' % gs_lr_tfidf.best_score_)

# 테스트 정확도 평가
clf = gs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


최적의 매개변수 조합: {'clf__C': 1.0, 'clf__penalty': 'l1', 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few',

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
48 fits failed out of a total of 96.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/usr/local/lib/pyth



---



In [107]:
import os
import gzip

if not os.path.isfile('movie_data.csv'):
  if not os.path.isfile('movie_data.csv.gz'):
    print('Please place a copy of the movie_data.csv.gz'
    'in this directory. You can obtain it by'
    'a) executing the code in the beginning of this'
    'notebook or b) by downloading it from GitHub:'
    'https://github.com/rickiepark/python-machine-learning-'
    'book-3rd-edition/blob/master/ch08/movie_data.csv.gz')

  else:
    with gzip.open('movie_data.csv.gz',
                   'rb') as in_f, \
                   open('movie_data.csv',
                        'wb') as out_f:
                        out_f.write(in_f.read())

In [108]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')

In [112]:
def tokenizer(text):
  text = re.sub('<[^>]*>','', text)
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
  text = re.sub('[\W]+', ' ', text.lower()) +\
  ' '.join(emoticons).replace('-','')

  tokenized = [w for w in text.split() if w not in stop]
  return tokenized

def stream_docs(path):
  with open(path, 'r', encoding='utf-8') as csv:
    next(csv) # 헤더 넘기기
    for line in csv:
      text, label = line[:-3], int(line[-2])
      yield text, label

next(stream_docs(path='movie_data.csv'))

def get_minibatch(doc_stream, size):
  docs, y = [], []
  try:
    for _ in range(size):
      text, label = next(doc_stream)
      docs.append(text)
      y.append(label)
  except StopIteration:
    return None, None
  return docs, y

In [121]:
from distutils.version import LooseVersion as Version

# 사이킷런 1.3버전을 사용하는 경우 'log'를 'log_loss'로 바꾸어 사용하세요.
clf = SGDClassifier(loss='log_loss', random_state=1)

doc_stream = stream_docs(path='movie_data.csv')

In [129]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
  X_train, y_train = get_minibatch(doc_stream, size=1000)
  if not X_train:
    break
  X_train = vect.transform(X_train)
  clf.partial_fit(X_train, y_train, classes=classes)
  pbar.update()

In [131]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

clf = clf.partial_fit(X_test, y_test)

TypeError: 'NoneType' object is not iterable